In [1]:
#to explore: the step make game lose = score should be lower? like: [-1,-1,0,0,1,0,0,0,0], next step should be 2, otherwise will lose for sure.
#to explore: should it be another model to explore all possible fastest lose situation? Or it is just put some noise will resolve
#to explore: add some noise into input/ output during training
#to explore: remove duplication on training data & score
#resolved - to explore: seperate game engine & cnn into seperate py file
#to explore: what should be a better DQ_ratio?
#to explore: create a chat to show training journey in a pic.
#to explore: Should that be using all pre-training data always everytime train the model??
#to explore: some scenarios AI always loss, may due to no noise on inout
#to explore: if j % 2 == 0:latest version, here needs to be re-visit, may impact final result.
#to explore: if possible to setup parameter, so that from Main script, it can directly call RL model for training with parameter as well.
#to explore: 

import os.path
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import torch.optim as optim
import numpy as np
from shutil import copyfile

#import py lib from a different folder: https://www.fir3net.com/Programming/Python/how-do-you-import-a-python-module-from-another-folder.html
#self created lib in py file format
from Model.Game_Agent import game_agent
from Model.DL_Network_Model import Net
from Model.Funtion_Bank import data_input


file_name_model_latest_version = 'Model/model_latest_version.pt'
file_name_model_last_version = 'Model/model_last_version.pt'
input_file_path = 'Data/training_data_input.csv'
score_file_path = 'Data/training_data_score.csv'
input_file_path_initiate = 'Data/training_data_input_initiate.csv'
score_file_path_initiate = 'Data/training_data_score_initiate.csv'
training_file_path = 'Data/training_data.csv'

torch.set_default_tensor_type('torch.cuda.FloatTensor')
#to explore: feel like this part can be improved in a way.
DQ_ratio = 0.70
epcho_no = 3000
steps_for_printing_out_loss = 1000
learning_rate = 0.1

def training_model():
    
    #1 means there is an existing model saved
    #0 means there is no model saved in folder
    
    if os.path.isfile(file_name_model_latest_version):
        indicator = 1
        input_data = data_input(input_file_path)
        score_data = data_input(score_file_path)
    else:
        indicator = 0
        input_data = data_input(input_file_path_initiate)
        score_data = data_input(score_file_path_initiate)
        #give some initial value is another option
        #input_data = torch.tensor([0,0,0,0,0,0,0,0,0], dtype=torch.float)
        #score_data = torch.tensor([0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11], dtype=torch.float)

    input = input_data
    target = score_data
    # create your optimizer
    net = Net()
    #to explore: feel like this part can be improved in a way.
    # 5 error functions with explainatioins(5 Regression Loss Functions All Machine Learners Should Know): https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0
    # HX: It should be alrighht to use MSE instead of MAE, as there is no outlier inside training data here
    criterion = nn.MSELoss()
    # to explore: visulize parameter updating process, as well as loss impact.
    # optim comparation between: SGD & Adam: https://medium.com/@Biboswan98/optim-adam-vs-optim-sgd-lets-dive-in-8dbf1890fbdc
    # Here should use Adam compared with SGD(need to explore other Oprim as well), but learning rate should be setup with some strategy. --------Pending
    optimizer = optim.SGD(net.parameters(), lr = learning_rate)
    # to explore: if if indicator == 0, any good start of parameter can be assigned?
    if indicator == 1:
        state_dict_last_version = torch.load('Model/model_latest_version.pt')['state_dict']
        net.load_state_dict(state_dict_last_version)
        #to explore function to load optimizer from existing model
        #optimizer.load_state_dict(torch.load('model_latest_version.pt')['optimizer'])
    # in your training loop:
    for i in range(1,epcho_no):
        #to explore a batch job for training
        #to explore how to save best performance model during 6000 epochs
        optimizer.zero_grad()   # zero the gradient buffers
        output = net(input)
        output[target == 0] = 0
        output[target == -1.1] = -1.1
        # difference between loss, error,criterion: https://datascience.stackexchange.com/questions/10250/what-is-the-difference-between-objective-error-criterion-cost-loss-fun
        # to explore: loss can be optimized as well - Pending
        loss = criterion(output, target)
        loss.backward()
        if i % (steps_for_printing_out_loss - 1) == 0:
            print(loss)
        optimizer.step()    # Does the update
    
    if indicator == 1:
        copyfile(file_name_model_latest_version, file_name_model_last_version)
    else:
        torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_last_version)
    torch.save({'state_dict': net.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_latest_version)
    
    export_output = output.clone()
    export_output = export_output.cpu().detach().numpy()
    np.savetxt('Data/output.csv',export_output,delimiter=',')

    export_target = target.clone()
    export_target = export_target.cpu().detach().numpy()
    np.savetxt('Data/target.csv',export_target,delimiter=',')



def main():
    tie = 0
    win = 0
    loss = 0
    
    #load model
    model_last_version = Net()
    state_dict_last_version = torch.load(file_name_model_last_version)['state_dict']
    model_last_version.load_state_dict(state_dict_last_version)
    
    model_latest_version = Net()
    state_dict_latest_version = torch.load(file_name_model_latest_version)['state_dict']
    model_latest_version.load_state_dict(state_dict_latest_version)
    
    training_data = data_input(training_file_path)
    #print(training_data)
    
    #initiallization
    #data_original = [1,0,0,-1,0,0,1,-1,0]
    #data_original = [0,1,0,0,0,0,-1,0,0]
    #data_original = [-1,1,0,1,1,-1,-1,-1,1]
    #data_original = [-1,1,1,1,1,-1,-1,-1,1]
    #data_original = [0,0,0,0,0,0,0,0,0]
    #torch_tensor = torch.tensor(data_original, dtype=torch.float)
    #print('torch_tensor 1st')
    #print(torch_tensor)
    #to explore: is that possible to perform below by batch, instead of 1 by 1?
    for x in range(0,82):
        torch_tensor = training_data[x].clone()
        for j in range(0,9):
            if j == 0:
                ###########another exceptional handling: if input without equal occurance of 1 & -1 - pending for coding
                game_a = game_agent(torch_tensor,0)
                if game_a.verify_result() == True:
                    print('not valid input, as game has been over')
                    break
                elif 0 not in torch_tensor:
                    print('input tie & with 9 value')
                    break
                else:
                    #initial
                    input_status = []
                    next_action_taken = []
                    score = []
                    #score = torch.zeros(9)
            torch_tensor_saved = torch_tensor.clone()
            input_status.append(torch_tensor_saved.cpu().numpy())
            #predict
            #if j % 2 == 1, it means to let last version take the first action
            if j % 2 == 0:
                next_vision = model_latest_version(torch_tensor)
            else:
                next_vision = model_last_version(torch_tensor)

            #print(next_vision)
            current_score = torch.zeros(9)
            #comprehance predict result to make impossible option as '-1'
            #here there is a big ?, how to aviod prediction result on position with existing point??
            for k in range(0,9):
                if torch_tensor[k] != 0:
                    next_vision[k] = -1.1
                    current_score[k] = -1.1
            next_step = next_vision.argmax()
            #save next action

            next_action_taken.append(next_step.cpu().numpy())
            score.append(current_score.cpu().numpy())
            #generate result
            torch_tensor[next_step] = 1
            game_a = game_agent(torch_tensor,0)
            # varify status of result
            # training data duplication should be avoid somewhere, also some inout, action, and different DQ score????????
            if game_a.verify_result() == True:
                #print(game_a.verify_result())
                #calculate DQ
                if j % 2 == 0:
                    win += 1
                else:
                    loss += 1
                #print('finished')
                ############ here as an enhancement on model, it could encourage model for a faster win strategy
                current_final_score = 1
                #update score, 
                #To explore: DQ learning score, any alogrithm wise enhancement can be done??
                for k in range(0, len(next_action_taken)):
                    score[-(k+1)][next_action_taken[-(k+1)]] = ((-1 * DQ_ratio)**(k))

                #####################break for what?
                break
            else:
                if 0 not in torch_tensor:
                    #print('tie')
                    tie = tie + 1
                    #any DQ score calculated for tie??????????
                    break
                else:
                    #generate next input
                    torch_tensor = torch_tensor * -1
        input_status_df = pd.DataFrame(input_status)
        input_status_df.to_csv('Data/training_data_input.csv', index=False, mode='a', header=False)
        next_action_taken_df = pd.DataFrame(next_action_taken)
        next_action_taken_df.to_csv('Data/training_data_next_action_taken.csv', index=False, mode='a', header=False)
        score_df = pd.DataFrame(score)
        score_df.to_csv('Data/training_data_score.csv', index=False, mode='a', header=False)
    print("tie" + str(tie))
    print("win" + str(win))
    print("loss" + str(loss))
    

    
if __name__ == "__main__":
    button = 1
    if button == 1:
        for loops in range(0,2):
            training_model()
            main()
    else:
        main()

tensor(0.0057, grad_fn=<MseLossBackward>)
tensor(0.0057, grad_fn=<MseLossBackward>)
tensor(0.0057, grad_fn=<MseLossBackward>)
tie35
win45
loss2
tensor(0.0056, grad_fn=<MseLossBackward>)
tensor(0.0056, grad_fn=<MseLossBackward>)
tensor(0.0056, grad_fn=<MseLossBackward>)
tie33
win46
loss3
